# Customer Segmentation

When it comes to finding out who your best customers are, the old RFM matrix principle is the best. 

RFM stands for *Recency*, *Frequency* and *Monetary.*   

It is a customer segmentation technique that uses past purchase behavior to divide customers into groups.  


### RFM Score Calculations  

**RECENCY (R)**: Days since last purchase  
**FREQUENCY (F)**: Total number of purchases  
**MONETARY VALUE (M)**: Total money this customer spent  

It is based on the marketing axiom that **80% of your business comes from 20% of your customers**.

RFM helps to identify customers who are more likely to respond to promotions by segmenting them into various categories.


### Import Packages

In [3]:
# !pip install xlrd
# import package
import pandas as pd
import datetime

# suppress error warnings
import warnings
warnings.filterwarnings('ignore')

# from google.colab import files

### Read Data

In [26]:
# load dataset
# data = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
data = pd.read_excel('Online Retail.xlsx')

# Define Sales Column
data['Sales'] = data['Quantity'] * data['UnitPrice']
data.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


### Create RFM Segments

These are the the only 4 things we need to build our RFM segments:
1. *customers* : feature that specifies your users,
2. *dates* : dates of transactions
3. *transactions* : transaction number
4. *prices* : price of amount sold

In [27]:
data.shape

(541909, 9)

In [28]:
customers = 'CustomerID'
dates = 'InvoiceDate'
transactions = 'InvoiceNo'
prices = 'Sales'

In [29]:
data['Description'] = data['Description'].str.strip()
data.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
data['InvoiceNo'] = data['InvoiceNo'].astype('str')
data = data[~data['InvoiceNo'].str.contains('C')]

In [30]:
#how does he know there are many duplicates?

data.Country.drop_duplicates()

0               United Kingdom
26                      France
197                  Australia
385                Netherlands
1109                   Germany
1236                    Norway
1404                      EIRE
5320               Switzerland
6421                     Spain
6608                    Poland
7134                  Portugal
7214                     Italy
7279                   Belgium
7986                 Lithuania
9783                     Japan
14938                  Iceland
20000          Channel Islands
20017                  Denmark
29732                   Cyprus
30079                   Sweden
34083                  Finland
34293                  Austria
38313                  Bahrain
50791                   Israel
69007                   Greece
69623                Hong Kong
70758                Singapore
72985                  Lebanon
89570     United Arab Emirates
100810            Saudi Arabia
103598          Czech Republic
119191                  Canada
152712  

In [31]:
country = "United Arab Emirates"

# Filter for a single country
data_country = data[data.Country == country]

In [32]:
data = data_country

NOW = datetime.datetime.now()

# RFM Feature Engineering
rfmTable = data.groupby(customers).agg({dates: lambda x: (NOW - x.max()).days, transactions: lambda x: len(x), prices: lambda x: x.sum()})
rfmTable[dates] = rfmTable[dates].astype(int)
rfmTable.rename(columns={dates: 'recency', 
                         transactions: 'frequency', 
                         prices: 'monetary_value'}, inplace=True)

In [33]:
rfmTable.head()

,recency,frequency,monetary_value
CustomerID,,,
12739.0,3005,38,1013.04
17829.0,3225,30,889.24


In [34]:
rfmTable.shape

(2, 3)

### Segment Users Based on Quantiles 

The easiest way to split metrics into segments is by using quartiles.  

1. This gives us a starting point for the detailed analysis.
2. 4 segments are easy to understand and explain.


In [35]:
quantiles = rfmTable.describe()
quantiles

,recency,frequency,monetary_value
count,2.000000,2.000000,2.00000
mean,3115.000000,34.000000,951.14000
std,155.563492,5.656854,87.53982
min,3005.000000,30.000000,889.24000
25%,3060.000000,32.000000,920.19000
50%,3115.000000,34.000000,951.14000
75%,3170.000000,36.000000,982.09000
max,3225.000000,38.000000,1013.04000


In [36]:
## RFM scorer
segmented_rfm = rfmTable.copy()

def RScore(x, p, d):
    if x <= d[p]['25%']:
        return 1
    elif x <= d[p]['50%']:
        return 2
    elif x <= d[p]['75%']: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p]['25%']:
        return 4
    elif x <= d[p]['50%']:
        return 3
    elif x <= d[p]['75%']: 
        return 2
    else:
        return 1

### Score Users

In [37]:
### Score each user
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency', quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))

### Merge Scores
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)
segmented_rfm['RFMScore'] = segmented_rfm['RFMScore'].map(int)

segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12739.0,3005,38,1013.04,1,1,1,111
17829.0,3225,30,889.24,4,4,4,444


In [38]:
data.to_csv('some_data.csv')

In [39]:

segmented_rfm.to_csv('some_file_name.csv')

# files.download('some_file_name.csv')

pd.read_csv('some_file_name.csv')

,CustomerID,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
0,12739.0,3005,38,1013.04,1,1,1,111
1,17829.0,3225,30,889.24,4,4,4,444


## Identify Customer Segments

### Who are the top 10 of our best customers?  
RFM Score of *111*.

In [40]:
segmented_rfm['RFMScore'].sort_values().unique()

array([111, 444], dtype=int64)

In [41]:
segmented_rfm[segmented_rfm['RFMScore']==111]

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12739.0,3005,38,1013.04,1,1,1,111


### Who are our Loyal Customers?  
Frequency score of *1*.

In [42]:
segmented_rfm[segmented_rfm['f_quartile']==1].sort_values('frequency', ascending=False).tail(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12739.0,3005,38,1013.04,1,1,1,111


### Who are our Big Spenders?  
Monetary value score of *1*.

In [43]:
segmented_rfm[segmented_rfm['m_quartile']==1].sort_values('monetary_value', ascending=False).tail(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12739.0,3005,38,1013.04,1,1,1,111


### Who are almost lost?  
Haven't purchased for some time but spent a lot and transacted a lot.  
RFM Score of *311*.

In [44]:
segmented_rfm[segmented_rfm['RFMScore']==311].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,


### Who are lost?  
Haven't purchased in the longest time but spent a lot and transacted a lot.  
RFM score of *411*.

In [45]:
segmented_rfm[segmented_rfm['RFMScore']==411].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,


### Who are the bottom 10 worst customers?  
RFM score of *444*.

In [47]:
segmented_rfm[segmented_rfm['RFMScore']==444].sort_values('monetary_value', ascending=False).tail(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
17829.0,3225,30,889.24,4,4,4,444


# Exercise: Do Customer Segmentation on a Different Dataset  
[link](https://query.data.world/s/oavhrdt2a4dhhg4agcbff6h3llnqsw) to dataset.